In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from datetime import date

import pandas as pd

In [2]:
app = Flask(__name__)
db_name = 'database/member_lists.db'

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + db_name

app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True

# this variable, db, will be used for all SQLAlchemy commands
db = SQLAlchemy(app)

In [3]:
class Member(db.Model): 
    __tablename__ = 'members_titles'
    id = db.Column(db.Integer, primary_key=True)
    title_id = db.Column(db.Integer)
    name_en = db.Column(db.String(255))
    name_ru = db.Column(db.String(255))
    max_ep = db.Column(db.Integer)
    from_usr = db.Column(db.String(100))
    to_usr = db.Column(db.String(100))

    def __init__(self, title_id, name_en, name_ru, max_ep, from_usr, to_usr):
        self.title_id = title_id
        self.name_en = name_en
        self.name_ru = name_ru
        self.max_ep = max_ep
        self.from_usr = from_usr
        self.to_usr = to_usr


In [4]:
db.create_all()

In [5]:
# delete all data from the table
db.session.query(Member).delete() 
db.session.commit()

In [8]:
# get data from SHIKIMORI
G_DRIVE_PATH = "G:\\My Drive\\ANIME_CHALLENGE"
from member_list import members
import yaml
from SHIKI_API import ShiNoAuth
a = ShiNoAuth()
trottle = 1
df_usr_dict = {}
for username in members:
        # read yaml file of member to list of ids
        # read from G_DRIVE
        # sleep(0.4)
        filename = f'{G_DRIVE_PATH}/{username}_list.yaml'
        with open(filename, 'r') as f:
            dl = yaml.load(f, Loader=yaml.FullLoader)
            # left only valid ids
            ids = [i for i in dl.values() if isinstance(i, int)]
        # get df w/ titles
        dfr = a.get_title_by_ids(ids, trottle)
        # name of user who assigned title
        dfr['from user'] = username
        # user who need to watch title
        dfr['to user'] = [key for key, val in dl.items() if isinstance(val, int)] 
        # df w/ all titles of member list
        df_usr_dict[username] = dfr
{i: list(df_usr_dict[i].values.tolist()) for i in members}

{'rengedead-': [[16,
   'Hachimitsu to Clover',
   'Мёд и клевер',
   24,
   'rengedead-',
   'AhresSsio_1'],
  [205, 'Samurai Champloo', 'Самурай Чамплу', 26, 'rengedead-', 'AhresSsio_2'],
  [10162, 'Usagi Drop', 'Брошенный кролик', 11, 'rengedead-', 'AhresSsio_3'],
  [32998, '91 Days', '91 день', 12, 'rengedead-', 'AhresSsio_4'],
  [11771,
   'Kuroko no Basket',
   'Баскетбол Куроко',
   25,
   'rengedead-',
   'AhresSsio_5'],
  [44387,
   'Sankaku Mado no Sotogawa wa Yoru',
   'Ночь по ту сторону треугольного окна',
   12,
   'rengedead-',
   'AhresSsio_6'],
  [40221, 'Kami no Tou', 'Башня Бога', 13, 'rengedead-', 'Elaeagnus_1'],
  [7054,
   'Kaichou wa Maid-sama!',
   'Президент студсовета — горничная!',
   26,
   'rengedead-',
   'Elaeagnus_2'],
  [18679, 'Kill la Kill', 'Убей или умри', 24, 'rengedead-', 'Elaeagnus_3'],
  [25183, 'Gangsta.', 'Гангста', 12, 'rengedead-', 'Elaeagnus_4'],
  [28223,
   'Death Parade',
   'Смертельный парад',
   12,
   'rengedead-',
   'Elaeagnus_5'],

In [9]:
# write data to SQLAlchemy
for i in members:
    df = df_usr_dict[i]
    for index, row in df.iterrows():
        # print(row.to_dict()['id'])
        record = Member(title_id=row.to_dict()['id'], 
            name_en=row.to_dict()['name_en'], 
            name_ru=row.to_dict()['name_ru'], 
            max_ep=row.to_dict()['ep_max'], 
            from_usr=row.to_dict()['from user'], 
            to_usr=row.to_dict()['to user']
            )
        db.session.add(record)
db.session.commit()
    

In [11]:
pd.read_sql(f'SELECT * FROM members_titles WHERE "from_usr" LIKE "{members[0]}%"',
            'sqlite:///' + db_name, 
            # index_col='id',
            )

,id,title_id,name_en,name_ru,max_ep,from_usr,to_usr
0,1,16,Hachimitsu to Clover,Мёд и клевер,24,rengedead-,AhresSsio_1
1,2,205,Samurai Champloo,Самурай Чамплу,26,rengedead-,AhresSsio_2
2,3,10162,Usagi Drop,Брошенный кролик,11,rengedead-,AhresSsio_3
3,4,32998,91 Days,91 день,12,rengedead-,AhresSsio_4
4,5,11771,Kuroko no Basket,Баскетбол Куроко,25,rengedead-,AhresSsio_5
5,6,44387,Sankaku Mado no Sotogawa wa Yoru,Ночь по ту сторону треугольного окна,12,rengedead-,AhresSsio_6
6,7,40221,Kami no Tou,Башня Бога,13,rengedead-,Elaeagnus_1
7,8,7054,Kaichou wa Maid-sama!,Президент студсовета — горничная!,26,rengedead-,Elaeagnus_2
8,9,18679,Kill la Kill,Убей или умри,24,rengedead-,Elaeagnus_3
9,10,25183,Gangsta.,Гангста,12,rengedead-,Elaeagnus_4
